# Converse API with tools

## Init

In [33]:
import json 
import math
import urllib
import boto3

## Define dependencies a tool error class


## Define a function to call Amazon Bedrock and return the response

We’re going to call Anthropic Claude 3 Sonnet using the converse method. We pass it a list of messages and a list of tools. We also set an output token limit and set the temperature to 0 to reduce the variability between calls (During development and testing, it can be preferable to set temperature higher for more variability in responses).
You can learn more about the Converse method [here](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html).


In [34]:
def call_bedrock(message_list, tool_list):
    session = boto3.Session()

    bedrock = session.client(service_name="bedrock-runtime")
    if tool_list: 
        response = bedrock.converse(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            messages=message_list,
            inferenceConfig={"maxTokens": 2000, "temperature": 0},
            toolConfig={"tools": tool_list},
        )
    else:
        response = bedrock.converse(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            messages=message_list,
            inferenceConfig={"maxTokens": 2000, "temperature": 0},
        )

    return response

## Add a function to handle tool use method calls

We’ll implement this function as a simple series of if/elif statements to call basic math functions or getting weather. 
Note that we're deliberately skipping the tangent tool so something interesting can happen!


### Weather tool

In [35]:
def get_weather(city: str):
    encoded_city = urllib.parse.quote(city)
    url = f"https://geocoding-api.open-meteo.com/v1/search?name={encoded_city}&count=1&language=en&format=json"
    with urllib.request.urlopen(url) as response:
        location_data = json.loads(response.read().decode())
        if not location_data["results"]:
            return {"error": "City not found"}

        lat = location_data["results"][0]["latitude"]
        lon = location_data["results"][0]["longitude"]

    weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m&daily=weather_code,temperature_2m_max,temperature_2m_min&timezone=auto"

    with urllib.request.urlopen(weather_url) as response:
        weather_data = json.loads(response.read().decode())

    current = weather_data["current"]
    daily = weather_data["daily"]

    weather_codes = {
        0: "Clear sky",
        1: "Mainly clear",
        2: "Partly cloudy",
        3: "Overcast",
        45: "Fog",
        48: "Depositing rime fog",
        51: "Light drizzle",
        53: "Moderate drizzle",
        55: "Dense drizzle",
        61: "Slight rain",
        63: "Moderate rain",
        65: "Heavy rain",
        71: "Slight snow fall",
        73: "Moderate snow fall",
        75: "Heavy snow fall",
        77: "Snow grains",
        80: "Slight rain showers",
        81: "Moderate rain showers",
        82: "Violent rain showers",
        85: "Slight snow showers",
        86: "Heavy snow showers",
        95: "Thunderstorm",
        96: "Thunderstorm with slight hail",
        99: "Thunderstorm with heavy hail",
    }
    response_core = {
        "temperature": current["temperature_2m"],
        "condition": weather_codes.get(current["weather_code"], "Unknown"),
        "humidity": current["relative_humidity_2m"],
        "wind_speed": current["wind_speed_10m"],
        "forecast_max": daily["temperature_2m_max"][0],
        "forecast_min": daily["temperature_2m_min"][0],
        "forecast_condition": weather_codes.get(daily["weather_code"][0], "Unknown"),
    }
    return response_core

In [36]:
get_weather("Paris")

{'temperature': 22.6,
 'condition': 'Clear sky',
 'humidity': 66,
 'wind_speed': 14.0,
 'forecast_max': 24.6,
 'forecast_min': 15.5,
 'forecast_condition': 'Moderate rain'}

### Defining the tools



We’re using a custom ToolError class to handle some of the potential things that can go wrong with tool use.

In [37]:
class ToolError(Exception):
    pass

In [38]:
def get_tool_result(tool_use_block):

    tool_use_name = tool_use_block["name"]

    print(f"Using tool {tool_use_name}")

    # Note: We're deliberately excluding tangent so something magical can happen
    if tool_use_name == "cosine":
        return math.cos(tool_use_block["input"]["x"])
    elif tool_use_name == "sine":
        return math.sin(tool_use_block["input"]["x"])
    elif tool_use_name == "divide_numbers":
        return tool_use_block["input"]["x"] / tool_use_block["input"]["y"]
    elif tool_use_name == "get_weather":
        return get_weather(tool_use_block["input"]["city"])
    else:
        raise ToolError(f"Invalid function name: {tool_use_name}")

## Add a function to handle LLM responses and determine if a follow-up tool call is needed
The LLM may return a combination of text and tool use content blocks in its response. We’ll look for tooUse content blocks, attempt to run the requested tools, and return a message with a toolResult block if a tool was used.



In [39]:
def handle_response(response_message):

    response_content_blocks = response_message["content"]

    follow_up_content_blocks = []

    for content_block in response_content_blocks:
        if "toolUse" in content_block:
            tool_use_block = content_block["toolUse"]

            try:
                tool_result_value = get_tool_result(tool_use_block)

                if tool_result_value is not None:
                    follow_up_content_blocks.append(
                        {
                            "toolResult": {
                                "toolUseId": tool_use_block["toolUseId"],
                                "content": [{"json": {"result": tool_result_value}}],
                            }
                        }
                    )

            except ToolError as e:
                follow_up_content_blocks.append(
                    {
                        "toolResult": {
                            "toolUseId": tool_use_block["toolUseId"],
                            "content": [{"text": repr(e)}],
                            "status": "error",
                        }
                    }
                )

    if len(follow_up_content_blocks) > 0:

        follow_up_message = {
            "role": "user",
            "content": follow_up_content_blocks,
        }

        return follow_up_message
    else:
        return None

## Add a function to run the request/response loop
This function will run a request / response loop until either the LLM stops requesting tool use or a maximum number of loops have run.


In [40]:
def run_loop(prompt, tool_list):
    MAX_LOOPS = 6
    loop_count = 0
    continue_loop = True

    message_list = [{"role": "user", "content": [{"text": prompt}]}]

    while continue_loop:
        response = call_bedrock(message_list, tool_list)

        response_message = response["output"]["message"]
        message_list.append(response_message)

        loop_count = loop_count + 1

        if loop_count >= MAX_LOOPS:
            print(f"Hit loop limit: {loop_count}")
            break

        follow_up_message = handle_response(response_message)

        if follow_up_message is None:
            # No remaining work to do, return final response to user
            continue_loop = False
        else:
            message_list.append(follow_up_message)

    return message_list

## Define the tools to use
We’re defining four tools for basic trigonometry functions, a division function and get weather.To deep dive into tool definition format, check [here](https://community.aws/content/2hWA16FSt2bIzKs0Z1fgJBwu589/generating-json-with-the-amazon-bedrock-converse-api).


In [41]:
tools = [
    {
        "toolSpec": {
            "name": "cosine",
            "description": "Calculate the cosine of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function.",
                        }
                    },
                    "required": ["x"],
                }
            },
        }
    },
    {
        "toolSpec": {
            "name": "sine",
            "description": "Calculate the sine of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function.",
                        }
                    },
                    "required": ["x"],
                }
            },
        }
    },
    {
        "toolSpec": {
            "name": "tangent",
            "description": "Calculate the tangent of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function.",
                        }
                    },
                    "required": ["x"],
                }
            },
        }
    },
    {
        "toolSpec": {
            "name": "divide_numbers",
            "description": "Divide x by y.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {"type": "number", "description": "The numerator."},
                        "y": {"type": "number", "description": "The denominator."},
                    },
                    "required": ["x", "y"],
                }
            },
        }
    },
    {
        "toolSpec": {
            "name": "get_weather",
            "description": "Get the weather for a city.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "city": {"type": "string", "description": "The city to get the weather for."},
                    },
                    "required": ["city"],
                }
            },
        }
    },
]

## Pass a prompt to start the loop


### Tangent
We’re asking Anthropic Claude to calculate the tangent of 7. We’re expecting a response with the calculated value.


But...

In [42]:
messages = run_loop("What is the tangent of 7 ?", [])

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=2))


MESSAGES:

[
  {
    "role": "user",
    "content": [
      {
        "text": "What is the tangent of 7 ?"
      }
    ]
  },
  {
    "role": "assistant",
    "content": [
      {
        "text": "The tangent function (tan) is a trigonometric function that gives the ratio of the opposite side to the adjacent side of a right-angled triangle.\n\nHowever, the tangent is only defined for angles between -90\u00b0 and 90\u00b0 (excluding -90\u00b0 and 90\u00b0). This is because for angles outside this range, the triangle would have an undefined opposite or adjacent side.\n\nThe value 7 by itself does not represent an angle in radians or degrees. So the tangent of 7 is undefined or meaningless in the context of trigonometry.\n\nIf you meant to ask for the tangent of an angle measured in radians or degrees, you would need to provide that angle value instead of just the number 7."
      }
    ]
  }
]


What if we we use tools ?

In [43]:
messages = run_loop("What is the tangent of 7 ?", tools)

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=2))

Using tool tangent
Using tool sine
Using tool cosine
Using tool divide_numbers

MESSAGES:

[
  {
    "role": "user",
    "content": [
      {
        "text": "What is the tangent of 7 ?"
      }
    ]
  },
  {
    "role": "assistant",
    "content": [
      {
        "text": "To calculate the tangent of 7, we can use the \"tangent\" tool:"
      },
      {
        "toolUse": {
          "toolUseId": "tooluse_rm1njQ83SSCXl6pvy4SXgw",
          "name": "tangent",
          "input": {
            "x": 7
          }
        }
      }
    ]
  },
  {
    "role": "user",
    "content": [
      {
        "toolResult": {
          "toolUseId": "tooluse_rm1njQ83SSCXl6pvy4SXgw",
          "content": [
            {
              "text": "ToolError('Invalid function name: tangent')"
            }
          ],
          "status": "error"
        }
      }
    ]
  },
  {
    "role": "assistant",
    "content": [
      {
        "text": "Oops, it seems the \"tangent\" tool is not available in this en

### Weather

We can ask Anthropic Claude to get the weather in Paris. We’re expecting a response with the current weather in Paris.

But...

In [44]:
messages = run_loop("What is Paris weather ?", [])

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=4))


MESSAGES:

[
    {
        "role": "user",
        "content": [
            {
                "text": "What is Paris weather ?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "Unfortunately, I don't have up-to-the-minute weather data for Paris. The weather can vary quite a bit in Paris depending on the time of year. Here are some general points about the typical weather in Paris:\n\n- Spring (March-May) - Mild temperatures, with highs around 15-20\u00b0C. Spring can be rainy at times.\n\n- Summer (June-August) - Warm to hot, with average highs around 24-27\u00b0C. Summer brings more sunshine but can also have occasional thunderstorms.\n\n- Fall (September-November) - Cool temperatures with highs of 15-20\u00b0C. Fall tends to be rainier than summer.\n\n- Winter (December-February) - Quite cold, with average highs around 7-9\u00b0C and lows near freezing. Winter brings clouds, rain and occasional snow flurrie

What if we we use tools ?

In [45]:
messages = run_loop("What is Paris weather ?", tools)

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=4))

Using tool get_weather

MESSAGES:

[
    {
        "role": "user",
        "content": [
            {
                "text": "What is Paris weather ?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "Okay, let me get the weather for Paris using the available tool:"
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_GKlTG6xnTsGnZYtzfN_-cg",
                    "name": "get_weather",
                    "input": {
                        "city": "Paris"
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": "tooluse_GKlTG6xnTsGnZYtzfN_-cg",
                    "content": [
                        {
                            "json": {
                                "result": {
                                    "temp

### Not needing tool question, but still using tools


In [46]:
messages = run_loop("Who is Barack Obama ?", tools)

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=4))


MESSAGES:

[
    {
        "role": "user",
        "content": [
            {
                "text": "Who is Barack Obama ?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "Barack Obama is an American politician who served as the 44th president of the United States from 2009 to 2017. Some key facts about him:\n\n- He was born on August 4, 1961 in Honolulu, Hawaii. He was the first African American president.\n\n- Before becoming president, he was a U.S. Senator representing Illinois from 2005 to 2008. \n\n- His presidential campaign calling for hope and change resonated with many Americans. He was elected in 2008 defeating John McCain and was re-elected in 2012 defeating Mitt Romney.\n\n- Major achievements as president include the Affordable Care Act (Obamacare) to reform healthcare, the economic stimulus package to address the Great Recession, the repeal of Don't Ask Don't Tell allowing LGBT people to ser